In [ ]:
#imported các thư viện cần thiết
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#đọc file dataset
test = pd.read_csv('../input/Testing.csv')
train = pd.read_csv('../input/Training.csv')




In [ ]:
test.head()

In [ ]:

test.shape

In [ ]:
train.shape

In [ ]:
df = pd.concat([train, test])

In [ ]:
df.shape

In [ ]:
#kiểm tra số lượng giá trị null trong bộ data
df.isnull().sum().sort_values(ascending=False)

In [ ]:
df.columns

In [ ]:
#kiểm tra tỉ lệ của mỗi bệnh trong bộ data
df['prognosis'].value_counts(normalize = True)

In [ ]:
#Các bệnh đều có tỉ lệ bằng nhau trong bộ data, có thể thấy qua sơ đồ dưới
df['prognosis'].value_counts(normalize = True).plot.bar()
plt.subplots_adjust(left = 0.9, right = 2 , top = 2, bottom = 1)

In [ ]:
#kiểm tra xem có các kiểu dữ liệu khác xuất hiện trong bộ data không 
df.dtypes.unique()

In [ ]:
#thống kê sự xuất hiện của các triệu chứng trong bộ data
for x in range(df.shape[1]):
    plt.subplot(7,22,x+1)
    plt.subplots_adjust(left = 0.5, right = 16 , top = 10, bottom = 0.5)
    sns.countplot(df[df.columns[x]]).set_title(df.columns[x],fontsize=23)

In [ ]:
#Kiểm qua mối quan hệ giữa các biến bằng biểu đồ tương quan
corr = df.corr()
mask = np.array(corr)
mask[np.tril_indices_from(mask)] = False
plt.subplots_adjust(left = 0.5, right = 16 , top = 20, bottom = 0.5)
sns.heatmap(corr, mask=mask,vmax=.9, square=True,annot=True, cmap="YlGnBu")

In [ ]:
#lấy ra 2 biến có độ tương quan cao và xem xét chúng thuộc giả thuyết null hay giả thuyết có mối quan hệ
#giả thuyết null cho rằng 2 biến này ko có mối quan hệ với nhau
pd.crosstab(df['cold_hands_and_feets'],df['weight_gain'])

In [ ]:
#Kiểm định chi-square độc lập được sử dụng để xác định xem có mối quan hệ có ý nghĩa thống kê giữa hai biến phân loại hay không. 
#Nó so sánh tần suất quan sát trong bảng phân phối tần suất với tần suất dự kiến dưới giả thuyết độc lập. 
#Nếu kết quả kiểm định cho ra giá trị p nhỏ hơn một ngưỡng ý nghĩa được chọn (thường là 0.05), thì cho thấy có mối quan hệ đáng kể giữa các biến.
from scipy.stats import chi2_contingency

In [ ]:
#vì giá trị p = 0.0 nhỏ hơn 0.05 nên 2 biến thuộc giả thuyết khác biệt.
#Cho thấy 2 biến thực sự khác nhau và có mối quan hệ đáng kể trong bộ dữ liệu
#Cho thấy sự tương quan giữa các triệu chứng trong quá trình chẩn đoán bệnh
chi2_contingency(pd.crosstab(df['cold_hands_and_feets'],df['weight_gain']))

In [ ]:
#tách các giá trị độc lập và phụ thuộc thành x và y
#x chứa các giá trị độc lập (triệu chứng)
#y chứa các giá trị phụ thuộc (kết quả chẩn đoán)
x = df.drop(['prognosis'],axis =1)
y = df['prognosis']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
#chia thành bộ train và test, với tỉ lệ là 0.33
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

In [ ]:
#import thuật toán naive bayes
from sklearn.naive_bayes import MultinomialNB

In [ ]:
#fit dữ liệu với model
mnb = MultinomialNB()
mnb = mnb.fit(x_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score



In [ ]:
y_pred = mnb.predict(x_test)

In [ ]:
accuracy_score(y_test,y_pred)

In [ ]:
#sử dụng cross-validation để đánh giá
from sklearn.model_selection import cross_val_score
scores = cross_val_score(mnb, x_test, y_test, cv=5)
print (scores)
print (scores.mean())

In [ ]:
#sử dụng 1 phương pháp cross khác là leave one out để đánh giá
from sklearn.model_selection import LeaveOneOut

In [ ]:
loo = LeaveOneOut()
scores = []
for train_index, test_index in loo.split(x):
    x_train, x_test = x.iloc[train_index], x.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    mnb.fit(x_train, y_train)
    y_pred = mnb.predict(x_test)
    score = accuracy_score(y_test, y_pred)
    scores.append(score)

In [ ]:
print(scores)
print(np.mean(scores))

In [ ]:
real_diseases = y_test.values

In [ ]:
# kiểm tra chính xác giữa giá trị dự đoán được và giá trị thực tế
for i in range(0, len(real_diseases)):
    if y_pred[i] == real_diseases[i]:
        print ('Pred: {0} Actual:{1}'.format(y_pred[i], real_diseases[i]))
    else:
        print('worng prediction')
        print ('Pred: {0} Actual:{1}'.format(y_pred[i], real_diseases[i]))

In [ ]:
#imported Kfold
from sklearn.model_selection import KFold

In [ ]:
## Hàm để xây dựng model với các thuật toán khác nhau cùng với K khác nhau sau này
def evaluate(train_data,kmax,algo):
    test_scores = {}
    train_scores = {}
    for i in range(2,kmax,2):
        kf = KFold(n_splits = i)
        sum_train = 0
        sum_test = 0
        data = df
        for train,test in kf.split(data):
            train_data = data.iloc[train,:]
            test_data = data.iloc[test,:]
            x_train = train_data.drop(["prognosis"],axis=1)
            y_train = train_data['prognosis']
            x_test = test_data.drop(["prognosis"],axis=1)
            y_test = test_data["prognosis"]
            algo_model = algo.fit(x_train,y_train)
            sum_train += algo_model.score(x_train,y_train)
            y_pred = algo_model.predict(x_test)
            sum_test += accuracy_score(y_test,y_pred)
        average_test = sum_test/i
        average_train = sum_train/i
        test_scores[i] = average_test
        train_scores[i] = average_train
        print("kvalue: ",i)
    return(train_scores,test_scores)   

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gbm = GradientBoostingClassifier()
nb = MultinomialNB()

In [ ]:
algo_dict = {'N_B' : nb}
algo_train_scores={}
algo_test_scores={}

In [ ]:

max_kfold = 11
for algo_name in algo_dict.keys():
    print(algo_name)
    tr_score,tst_score = evaluate(df,max_kfold,algo_dict[algo_name])
    algo_train_scores[algo_name] = tr_score
    algo_test_scores[algo_name] = tst_score
print(algo_train_scores)
print(algo_test_scores)

In [ ]:
df_test = pd.DataFrame(algo_test_scores)
df_train = pd.DataFrame(algo_train_scores)

In [ ]:
df_test.plot(grid = 1)
plt.show()
plt.grid()

In [ ]:
#Chọn k = 2 để build model
test_scores={}
train_scores={}
for i in range(2,4,2):
    kf = KFold(n_splits = i)
    sum_train = 0
    sum_test = 0
    data = df
    for train,test in kf.split(data):
        train_data = data.iloc[train,:]
        test_data = data.iloc[test,:]
        x_train = train_data.drop(["prognosis"],axis=1)
        y_train = train_data['prognosis']
        x_test = test_data.drop(["prognosis"],axis=1)
        y_test = test_data["prognosis"]
        algo_model = nb.fit(x_train,y_train)
        sum_train += nb.score(x_train,y_train)
        y_pred = nb.predict(x_test)
        sum_test += accuracy_score(y_test,y_pred)
    average_test = sum_test/i
    average_train = sum_train/i
    test_scores[i] = average_test
    train_scores[i] = average_train
    print("kvalue: ",i)  

In [ ]:
print(train_scores)
print(test_scores)

In [ ]:
#Lưu model thành file
from sklearn.externals import joblib
joblib.dump(nb,'trained_model')

In [ ]:
#Test
a = list(range(2,134))
i_name  = (input('Enter your name :'))
i_age = (int(input('Enter your age:')))
for i in range(len(x.columns)):
    print(str(i+1+1) + ":", x.columns[i])
choices = input('Enter the Serial no.s which is your Symptoms are exist:  ')
b = [int(x) for x in choices.split()]
count = 0
while count < len(b):
    item_to_replace =  b[count]
    replacement_value = 1
    indices_to_replace = [i for i,x in enumerate(a) if x==item_to_replace]
    count += 1
    for i in indices_to_replace:
        a[i] = replacement_value
a = [0 if x !=1 else x for x in a]
y_diagnosis = nb.predict([a])
y_pred_2 = nb.predict_proba([a])
print(('Name of the infection = %s , confidence score of : = %s') %(y_diagnosis[0],y_pred_2.max()* 100),'%' )
print(('Name = %s , Age : = %s') %(i_name,i_age))